In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
df_9 = pd.read_csv('On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_9_.csv')
df_10 = pd.read_csv('On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_10_.csv')
df_11 = pd.read_csv('On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_11_.csv')
df_12 = pd.read_csv('On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_12_.csv')


C:\Users\varun\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (49,77,78,85) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\varun\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (49,78,85) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\varun\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (49,70,77,78,85) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\varun\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (49,77,78,85,86,93) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, comp

In [3]:
def revise_df(df):
    
    Data= df[df['Reporting_Airline']=='NK']
    
    data14 = pd.DataFrame(Data,columns=['ArrTime','CRSArrTime','DepTime','CRSDepTime'])
    data14 = pd.to_numeric(data14.stack(), errors='coerce').fillna(0)
    df2= (pd.to_timedelta(data14 // 100, unit='hours') + pd.to_timedelta(data14 % 100, unit='T')).unstack()
#     ddf = Data.dropna(axis=0, subset=['ArrTime','CRSArrTime','DepTime','CRSDepTime'])
    ddf = Data
    df_merged = ddf.merge(df2, how='outer', left_index=True, right_index=True)
    
    df_merged.drop(labels=['Quarter','Unnamed: 109','ArrTime_x','CRSArrTime_x',
                           'DepTime_x','CRSDepTime_x'],inplace=True,axis=1)
    
    df_merged.reset_index(drop=True, inplace=True)
    
    
    df_merged['FlightDate']=pd.to_datetime(df_merged['FlightDate'])
#     df_merged=df_merged[pd.notnull(df_merged["ArrTime_y"])]
    df_merged.loc[df_merged['ArrTime_y'] > df_merged['CRSArrTime_y'], 'Delayed'] = 1
    df_merged.loc[df_merged['ArrTime_y'] <= df_merged['CRSArrTime_y'], 'Delayed'] = 0

    df_merged['Delayed']=df_merged['Delayed'].fillna(0)
    
    df_merged['Delayed']=df_merged['Delayed'].astype('int64')
    df_merged['Diverted']=df_merged['Diverted'].astype('int64')
    df_merged['Cancelled']=df_merged['Cancelled'].astype('int64')
    
    df_merged = df_merged.rename(columns={"ArrTime_y": "ArrTime", "CRSArrTime_y": "CRSArrTime",
                                          "DepTime_y": "DepTime", "CRSDepTime_y": "CRSDepTime"})
    
    return df_merged

In [4]:
df_merged_9 = revise_df(df_9)
df_merged_10 = revise_df(df_10)
df_merged_11 = revise_df(df_11)
df_merged_12 = revise_df(df_12)


In [5]:
df_merged_all = pd.concat([df_merged_9, df_merged_10, df_merged_11, df_merged_12])

In [6]:
df_reorder_all = df_merged_all[(list(df_merged_all.columns[0:10])
                                +list(df_merged_all.columns[-5:-1])
                                +[df_merged_all.columns[-1]]
                                +list(df_merged_all.columns[10: -5]))]

In [7]:
df_reorder_all.to_csv("reporting_carrier_ontime_performance_final.csv")

----------
# just check

In [8]:
pd.unique(df_merged_all["Cancelled"])

array([1, 0], dtype=int64)

In [9]:
pd.unique(df_merged_9["Cancelled"])

array([1, 0], dtype=int64)

In [10]:
df_merged_9.shape[0]+df_merged_10.shape[0]+df_merged_11.shape[0]+df_merged_12.shape[0]

68407

In [11]:
(df_9[df_9['Reporting_Airline']=='NK'].shape[0]
 +df_10[df_10['Reporting_Airline']=='NK'].shape[0]
 +df_11[df_11['Reporting_Airline']=='NK'].shape[0]
 +df_12[df_12['Reporting_Airline']=='NK'].shape[0]
)

68407

In [12]:
pd.unique(df_merged_all["Delayed"])

array([0, 1], dtype=int64)

In [13]:
pd.unique(df_merged_all["Diverted"])

array([0, 1], dtype=int64)

In [14]:
pd.unique(df_merged_all["Cancelled"])

array([1, 0], dtype=int64)